In [2]:
# ============================================
# 병목 임계치 자동 결정 & 레이블링 유틸
#  - y_reg: 시점별 병목 강도 = rowwise max(queue_* 컬럼)
#  - 여러 기준(상위p%, 평균+2σ, IQR, 영향도기반)을 비교하고 추천
#  - 기본 정의는 HTML 문서의 P90(상위 10%) 기준과 일치
# ============================================

import numpy as np
import pandas as pd

# -------- 0) 설정 --------
CSV_PATH = "Final Results Extended.csv"  # 경로 맞춰주세요
QUEUE_KEY = "queue"  # 'queue'가 들어간 모든 컬럼을 Queue 후보로 간주(대소문자 무관)
TARGET_TOTAL = "c_TotalProducts"  # 생산량 타깃(있으면 영향도 기반 계산에 활용)
PERCENTILE = 90  # HTML에서 사용한 기본 병목 기준: 상위 10% (=P90)

# -------- 1) 로드 & 컬럼 정규화 --------
df = pd.read_csv(CSV_PATH, low_memory=False)
df.columns = (
    df.columns
      .str.replace("__", "_", regex=False)
      .str.replace(" ", "", regex=False)
)

# -------- 2) Queue 컬럼 수집 & y_reg 계산 --------
queue_cols = [c for c in df.columns if QUEUE_KEY.lower() in c.lower()]
if not queue_cols:
    raise ValueError("Queue 관련 컬럼을 찾지 못했습니다. 컬럼명에 'queue'가 포함되어 있는지 확인하세요.")

# 수치 변환(에러는 NaN) 후 y_reg = 각 행에서 queue_cols의 최대값
df_queue = df[queue_cols].apply(pd.to_numeric, errors="coerce")
y_reg = df_queue.max(axis=1)
df["_y_reg_bottleneck_strength"] = y_reg

# -------- 3) 후보 임계치 계산 함수들 --------
def threshold_by_percentile(series, p=90):
    thr = np.nanpercentile(series, p)
    return thr, f"percentile_{p}"

def threshold_by_mean_std(series, k=2.0):
    mu, sigma = np.nanmean(series), np.nanstd(series)
    thr = mu + k * sigma
    return thr, f"mean_plus_{k}sigma"

def threshold_by_iqr(series, k=1.5):
    q1, q3 = np.nanpercentile(series, 25), np.nanpercentile(series, 75)
    iqr = q3 - q1
    thr = q3 + k * iqr
    return thr, f"iqr_{k}x"

def threshold_by_impact(series, y_total, top_rate_candidates=(0.05, 0.1, 0.15, 0.2)):
    """
    '영향도 기반': 여러 상위비율(top_rate)로 이진 레이블을 만들고,
    각 레이블이 TotalProducts와 갖는 음의 상관(|corr| 최대)을 찾습니다.
    (병목이 심하면 생산량이 낮아진다는 전제하에 음의 상관을 선호)
    """
    res = []
    s = series.copy()
    for r in top_rate_candidates:
        p = 100 * (1 - r)  # 예: r=0.1 -> P90
        thr = np.nanpercentile(s, p)
        lab = (s >= thr).astype(int)
        if y_total is not None and pd.api.types.is_numeric_dtype(y_total):
            corr = pd.Series(y_total).corr(lab)  # 피어슨
        else:
            corr = np.nan
        res.append((thr, f"impact_top{int(r*100)}%", corr))
    # corr이 nan이 아닌 것 중 '음의 상관(작을수록 좋음)' 절대값 최댓값 선택
    candidates = [x for x in res if not np.isnan(x[2])]
    if candidates:
        # 음의 상관 절대값이 가장 큰 항목을 우선, 동률이면 상위비율 작은 것 선호
        candidates.sort(key=lambda x: (abs(min(0, x[2])), -float(x[1].split('top')[-1].rstrip('%'))), reverse=True)
        return candidates[0]
    else:
        # 생산량 타깃이 없거나 상관 계산 불가하면 기본 P90로 fallback
        p = 90
        thr = np.nanpercentile(series, p)
        return thr, f"impact_fallback_percentile_{p}", np.nan

# -------- 4) 모든 후보 임계치 산출 --------
thr_pctl, name_pctl = threshold_by_percentile(y_reg, PERCENTILE)
thr_mean2s, name_mean2s = threshold_by_mean_std(y_reg, 2.0)
thr_iqr, name_iqr = threshold_by_iqr(y_reg, 1.5)
thr_imp, name_imp, corr_imp = threshold_by_impact(y_reg, df.get(TARGET_TOTAL, None))

# -------- 5) 각 기준으로 레이블 생성 & 통계 비교 --------
def label_by_threshold(series, thr):
    return (series >= thr).astype(int)

summaries = []
for thr, tag in [
    (thr_pctl, name_pctl),
    (thr_mean2s, name_mean2s),
    (thr_iqr, name_iqr),
    (thr_imp, name_imp),
]:
    lab = label_by_threshold(y_reg, thr)
    pos_rate = lab.mean()
    mu_pos = y_reg[lab == 1].mean()
    mu_neg = y_reg[lab == 0].mean()
    # 생산량 영향(있을 때만)
    if TARGET_TOTAL in df.columns and pd.api.types.is_numeric_dtype(df[TARGET_TOTAL]):
        corr_total = df[TARGET_TOTAL].corr(lab)  # 음수가 클수록(절댓값) '병목이면 생산량↓' 가설에 부합
    else:
        corr_total = np.nan
    summaries.append({
        "criterion": tag,
        "threshold": float(thr),
        "positive_rate": float(pos_rate),
        "y_reg_mean_pos": float(mu_pos),
        "y_reg_mean_neg": float(mu_neg),
        "corr_total_vs_label": float(corr_total) if not np.isnan(corr_total) else None,
    })

report = pd.DataFrame(summaries).sort_values("positive_rate")
print(report.to_string(index=False))

# -------- 6) 추천 로직 --------
# 1) 생산량과의 상관이 계산 가능하면: corr_total_vs_label의 '음의 상관 절댓값'이 가장 큰 기준 추천
# 2) 아니면: 분포 상단을 안정적으로 잡는 IQR 또는 P90 중에서 양성비(positive_rate)가 과도하지 않은 기준 선택
def recommend_threshold(df_report: pd.DataFrame):
    rep = df_report.copy()
    if "corr_total_vs_label" in rep and rep["corr_total_vs_label"].notna().any():
        rep["_score"] = rep["corr_total_vs_label"].apply(lambda c: abs(min(0, c)) if pd.notna(c) else 0.0)
        best = rep.sort_values(["_score", "positive_rate"], ascending=[False, True]).iloc[0]
        reason = "생산량(c_TotalProducts)과의 음의 상관 절댓값이 가장 큼(병목일수록 생산량 감소 가설 부합)"
    else:
        # 생산량이 없거나 상관이 nan이면: IQR 우선, 그다음 P90
        pref = rep.set_index("criterion")
        if "iqr_1.5x" in pref.index:
            best = pref.loc["iqr_1.5x"]
            reason = "분포의 상단 이상치를 안정적으로 포착(IQR 1.5×)"
        else:
            best = pref.loc["percentile_90"] if "percentile_90" in pref.index else pref.iloc[0]
            reason = "HTML 기본 정의(P90) 준수 및 보수적 상단 포착"
    return best.to_dict(), reason

best, why = recommend_threshold(report)
print("\n[추천 임계치]")
for k, v in best.items():
    print(f"- {k}: {v}")
print(f"- 사유: {why}")

# -------- 7) 최종 레이블 y_cls 생성(추천 기준 기반) --------
thr_final = best["threshold"]
df["_y_cls_bottleneck"] = (df["_y_reg_bottleneck_strength"] >= thr_final).astype(int)

# -------- 8) 결과 예시 확인 --------
print("\n라벨 분포:", df["_y_cls_bottleneck"].value_counts(normalize=True).round(3).to_dict())

# (선택) CSV로 저장
# df[["_y_reg_bottleneck_strength", "_y_cls_bottleneck"]].to_csv("bottleneck_labels.csv", index=False)
# report.to_csv("bottleneck_threshold_report.csv", index=False)

         criterion  threshold  positive_rate  y_reg_mean_pos  y_reg_mean_neg  corr_total_vs_label
mean_plus_2.0sigma 370.518893       0.043798      487.674463      190.416677             0.048130
      impact_top5% 357.503448       0.050002      472.305613      189.284624             0.051439
          iqr_1.5x 326.646461       0.069771      435.099476      186.060268             0.058576
     percentile_90 296.877103       0.100003      397.479159      181.875038             0.068823

[추천 임계치]
- criterion: mean_plus_2.0sigma
- threshold: 370.51889270906213
- positive_rate: 0.04379842624136995
- y_reg_mean_pos: 487.6744628260196
- y_reg_mean_neg: 190.41667724451975
- corr_total_vs_label: 0.04813048349001452
- _score: 0
- 사유: 생산량(c_TotalProducts)과의 음의 상관 절댓값이 가장 큼(병목일수록 생산량 감소 가설 부합)

라벨 분포: {0: 0.956, 1: 0.044}
